In [1]:
import pandas as pd

# Read the CSV with proper encoding
df = pd.read_csv('../data/raw/wework_data.csv', sep=";")


In [2]:
df

,URL,Address,City,Country
0,https://www.wework.com/buildings/seoul-square-...,"13F, 416 Hangang-daero Jung-gu Seoul, Seoul 04637",Seoul,South Korea
1,https://www.wework.com/buildings/university-pa...,"3300 N Interstate 35 Suite 700 Austin, TX 78705",Austin TX,USA
2,https://www.wework.com/buildings/urban-escape-...,"Regeringsgatan 29, Stockholms Stockholm, Stock...",Stockholm,Sweden
3,https://www.wework.com/buildings/wallarkaden--...,"PilgrimstraÃŸe 6 KÃ¶ln, Nordrhein-Westfalen 50674",Cologne,Germany
4,https://www.wework.com/buildings/25-king-stree...,"25 King Street Bowen Hills Brisbane, QLD 4006",Brisbane QLD,Australia
...,...,...,...,...
427,https://www.wework.com/buildings/222-broadway-...,"222 Broadway 22nd Floor New York, NY 10038",New-york-city NY,USA
428,https://www.wework.com/buildings/neue-schonhau...,"Neue SchÃ¶nhauser StraÃŸe 3-5 Berlin, BE 10178",Berlin,Germany
429,https://www.wework.com/buildings/400-capitol-m...,"400 Capitol Mall Sacramento, CA 95814",Sacramento CA,USA
430,https://www.wework.com/buildings/sinsa--seoul,"1F, 586, Gangnam-daero Gangnam-gu Seoul, Seoul...",Seoul,South Korea


In [3]:
#df["Address"] = df["Address"].apply(lambda x: x.encode("utf-8", errors="replace").decode("utf-8") if isinstance(x, str) else x)

In [4]:
#df["Address"] = df["Address"].apply(lambda x: x.encode("utf-8", errors="ignore").decode("utf-8") if isinstance(x, str) else x)

In [5]:
from ftfy import fix_text

df["Address"] = df["Address"].apply(lambda x: fix_text(x) if isinstance(x, str) else x)

In [6]:
df

,URL,Address,City,Country
0,https://www.wework.com/buildings/seoul-square-...,"13F, 416 Hangang-daero Jung-gu Seoul, Seoul 04637",Seoul,South Korea
1,https://www.wework.com/buildings/university-pa...,"3300 N Interstate 35 Suite 700 Austin, TX 78705",Austin TX,USA
2,https://www.wework.com/buildings/urban-escape-...,"Regeringsgatan 29, Stockholms Stockholm, Stock...",Stockholm,Sweden
3,https://www.wework.com/buildings/wallarkaden--...,"Pilgrimstraße 6 Köln, Nordrhein-Westfalen 50674",Cologne,Germany
4,https://www.wework.com/buildings/25-king-stree...,"25 King Street Bowen Hills Brisbane, QLD 4006",Brisbane QLD,Australia
...,...,...,...,...
427,https://www.wework.com/buildings/222-broadway-...,"222 Broadway 22nd Floor New York, NY 10038",New-york-city NY,USA
428,https://www.wework.com/buildings/neue-schonhau...,"Neue Schönhauser Straße 3-5 Berlin, BE 10178",Berlin,Germany
429,https://www.wework.com/buildings/400-capitol-m...,"400 Capitol Mall Sacramento, CA 95814",Sacramento CA,USA
430,https://www.wework.com/buildings/sinsa--seoul,"1F, 586, Gangnam-daero Gangnam-gu Seoul, Seoul...",Seoul,South Korea


In [7]:
# import requests

# # Function to get latitude and longitude using Geocode.Farm
# def get_lat_lon_geocode_farm(address):
#     try:
#         url = f"https://www.geocode.farm/v3/json/forward/?addr={address}&key=free-tier-key"
#         response = requests.get(url)
#         data = response.json()
#         if data["geocoding_results"]["STATUS"]["status"] == "SUCCESS":
#             location = data["geocoding_results"]["RESULTS"][0]["geometry"]["location"]
#             return pd.Series([location["lat"], location["lng"]])
#         return pd.Series([None, None])
#     except Exception:
#         return pd.Series([None, None])

# # Apply function to Address column
# df[["Latitude", "Longitude"]] = df["Address"].apply(get_lat_lon_geocode_farm)

In [8]:
import pycountry

# Function to refine and correct country codes
def refine_country_code(country_name):
    if not isinstance(country_name, str):
        return None  # Handle non-string values safely

    country_name = country_name.strip().lower()  # Normalize the country name
    corrections = {
        "uae": "AE",  # United Arab Emirates
        "united arab emirates": "AE",
        "uk": "GB",  # United Kingdom
        "united kingdom": "GB",
        "usa": "US",
        "united states": "US",
        "south korea": "KR",
        "north korea": "KP",
    }
    
    # Check if the country is in the manual corrections list
    if country_name in corrections:
        return corrections[country_name]
    
    # Try to get the country code using pycountry
    try:
        return pycountry.countries.lookup(country_name).alpha_2
    except LookupError:
        return None  # Return None if country not found

# Apply the refined function to update the 'country_code' column
df["country_code"] = df["Country"].apply(refine_country_code)

# Check if any country_code values are still null
missing_country_codes = df[df["country_code"].isnull()]

# Print missing country codes (if any)
print(f"Missing country codes:\n{missing_country_codes}")


Missing country codes:
                                                   URL  \
365  https://www.wework.com/buildings/jr-sendai-eas...   

                                               Address  \
365  1-Chome-1-1 Tsutsujigaoka, Miyagino-Ku Sendai,...   

                                            City Country country_code  
365  Sendai-d440a588-7f29-494e-8bee-84ba5e85f95d       0         None  


In [9]:
# Display rows where 'country_code' is null
missing_country_codes = df[df["country_code"].isnull()]
missing_country_codes

,URL,Address,City,Country,country_code
365,https://www.wework.com/buildings/jr-sendai-eas...,"1-Chome-1-1 Tsutsujigaoka, Miyagino-Ku Sendai,...",Sendai-d440a588-7f29-494e-8bee-84ba5e85f95d,0,None


In [10]:
df.isnull().sum()

URL             0
Address         0
City            0
Country         0
country_code    1
dtype: int64

In [11]:
# Drop rows where 'country_code' is null
df = df.dropna(subset=["country_code"])

# Reset index after dropping rows
df.reset_index(drop=True, inplace=True)

In [12]:
# Convert all column names to lowercase
df.columns = df.columns.str.lower()

# Drop the 'url' column if it exists
if "url" in df.columns:
    df = df.drop(columns=["url"])

# Display the cleaned DataFrame
df.head() # Show the first few rows to confirm changes

,address,city,country,country_code
0,"13F, 416 Hangang-daero Jung-gu Seoul, Seoul 04637",Seoul,South Korea,KR
1,"3300 N Interstate 35 Suite 700 Austin, TX 78705",Austin TX,USA,US
2,"Regeringsgatan 29, Stockholms Stockholm, Stock...",Stockholm,Sweden,SE
3,"Pilgrimstraße 6 Köln, Nordrhein-Westfalen 50674",Cologne,Germany,DE
4,"25 King Street Bowen Hills Brisbane, QLD 4006",Brisbane QLD,Australia,AU


In [13]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# Initialize geolocator
geolocator = Nominatim(user_agent="geoapi")

# Function to get latitude and longitude
def get_lat_lon(city):
    try:
        location = geolocator.geocode(city, timeout=10)
        if location:
            return pd.Series([location.latitude, location.longitude])
        else:
            return pd.Series([None, None])
    except GeocoderTimedOut:
        return pd.Series([None, None])

# Apply function to DataFrame
df[['latitude', 'longitude']] = df['city'].apply(get_lat_lon)

In [14]:
# Convert 'latitude' and 'longitude' to float
df["latitude"] = pd.to_numeric(df["latitude"], errors="coerce")
df["longitude"] = pd.to_numeric(df["longitude"], errors="coerce")

In [15]:
df

,address,city,country,country_code,latitude,longitude
0,"13F, 416 Hangang-daero Jung-gu Seoul, Seoul 04637",Seoul,South Korea,KR,37.566679,126.978291
1,"3300 N Interstate 35 Suite 700 Austin, TX 78705",Austin TX,USA,US,30.271129,-97.743700
2,"Regeringsgatan 29, Stockholms Stockholm, Stock...",Stockholm,Sweden,SE,59.325117,18.071093
3,"Pilgrimstraße 6 Köln, Nordrhein-Westfalen 50674",Cologne,Germany,DE,43.721828,0.977496
4,"25 King Street Bowen Hills Brisbane, QLD 4006",Brisbane QLD,Australia,AU,-27.468968,153.023499
...,...,...,...,...,...,...
426,"222 Broadway 22nd Floor New York, NY 10038",New-york-city NY,USA,US,40.712728,-74.006015
427,"Neue Schönhauser Straße 3-5 Berlin, BE 10178",Berlin,Germany,DE,52.510885,13.398937
428,"400 Capitol Mall Sacramento, CA 95814",Sacramento CA,USA,US,38.581061,-121.493895
429,"1F, 586, Gangnam-daero Gangnam-gu Seoul, Seoul...",Seoul,South Korea,KR,37.566679,126.978291


In [16]:
df.dtypes

address          object
city             object
country          object
country_code     object
latitude        float64
longitude       float64
dtype: object

In [17]:
df = df[["country","city","address","latitude","longitude","country_code"]]

In [18]:
# Read the CSV file with UTF-8-SIG encoding
# df = pd.read_csv("sql_wework_fixed.csv", encoding="utf-8-sig")

# Save the CSV file again with proper UTF-8 encoding (without BOM)
# df.to_csv("sql_wework_cleaned.csv", index=False, encoding="utf-8")


In [20]:
from unidecode import unidecode

# Normalize address column
df["address"] = df["address"].apply(lambda x: unidecode(str(x)))

In [22]:
df

,country,city,address,latitude,longitude,country_code
0,South Korea,Seoul,"13F, 416 Hangang-daero Jung-gu Seoul, Seoul 04637",37.566679,126.978291,KR
1,USA,Austin TX,"3300 N Interstate 35 Suite 700 Austin, TX 78705",30.271129,-97.743700,US
2,Sweden,Stockholm,"Regeringsgatan 29, Stockholms Stockholm, Stock...",59.325117,18.071093,SE
3,Germany,Cologne,"Pilgrimstrasse 6 Koln, Nordrhein-Westfalen 50674",43.721828,0.977496,DE
4,Australia,Brisbane QLD,"25 King Street Bowen Hills Brisbane, QLD 4006",-27.468968,153.023499,AU
...,...,...,...,...,...,...
426,USA,New-york-city NY,"222 Broadway 22nd Floor New York, NY 10038",40.712728,-74.006015,US
427,Germany,Berlin,"Neue Schonhauser Strasse 3-5 Berlin, BE 10178",52.510885,13.398937,DE
428,USA,Sacramento CA,"400 Capitol Mall Sacramento, CA 95814",38.581061,-121.493895,US
429,South Korea,Seoul,"1F, 586, Gangnam-daero Gangnam-gu Seoul, Seoul...",37.566679,126.978291,KR


In [24]:
df.to_csv("../data/clean/sql_wework.csv", index=False, sep=";")